In [ ]:
import os, boto3, time

In [ ]:

# Retrieve the AWS credentials and region from environment variables
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
region_name = os.getenv("AWS_DEFAULT_REGION")
region_name = 'us-west-q'

print(f"AWS Access Key ID: {aws_access_key_id}")
print(f"AWS Secret Access Key: {aws_secret_access_key}")
print(f"Region: {region_name}")

db_username = 'admin'
db_password = 'password_r00t'
db_subset_group = 'vpc-creation-n-interaction-study-rds-subnet-group'
db_cluster_id = 'rds-management-cluster'

In [ ]:
rds_client = boto3.client('rds')

In [ ]:
# create the DB cluster

try:
    response = rds_client.describe_db_clusters(
        DBClusterIdentifier=db_cluster_id,
    )
    print(f"DB Cluster {db_cluster_id} already exists, skipping creation")
except rds_client.exceptions.DBClusterNotFoundFault:
    print(f"Creating DB Cluster {db_cluster_id}")
    response = rds_client.create_db_cluster(
        DBClusterIdentifier=db_cluster_id,
        Engine="aurora-mysql",
        EngineVersion="5.7.mysql_aurora.2.11.1",
        MasterUsername=db_username,
        MasterUserPassword=db_password,
        DatabaseName="rds_management",
        DBSubnetGroupName=db_subset_group,
        EngineMode="provisioned"
    )

    print(f"DB Cluster {db_cluster_id} created")
    
    while True:
        response = rds_client.describe_db_clusters(
            DBClusterIdentifier=db_cluster_id,
        )
        status = response['DBClusters'][0]['Status']
        print(f"DB Cluster status: {status}")
        if status == 'available':
            break
        time.sleep(10)

In [ ]:
# delete the DB cluster
try:
    response = rds_client.delete_db_cluster(
        DBClusterIdentifier=db_cluster_id,
        SkipFinalSnapshot=True
    )
    print(f"DB Cluster {db_cluster_id} deleted")
except rds_client.exceptions.DBClusterNotFoundFault:
    print(f"DB Cluster {db_cluster_id} does not exist, skipping deletion")
    